## Importing libraries and dependencies

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

## Loading Pipeline

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


## Loading Blog

In [ ]:
URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"

In [ ]:
url = 'https://medium.com/aimluae/tensorflow-vs-pytorch-which-will-be-the-top-deep-learning-framework-in-2022-a3488635029'

In [ ]:
r = requests.get(url)

In [ ]:
r.text

'<!doctype html><html lang="en"><head><title data-rh="true">Tensorflow vs PyTorch: Which Will Be The Top Deep Learning Framework In 2022? | by Salman Khan | AIML UAE | Jun, 2022 | Medium</title><meta data-rh="true" charset="utf-8"/><meta data-rh="true" name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1"/><meta data-rh="true" name="theme-color" content="#000000"/><meta data-rh="true" name="twitter:app:name:iphone" content="Medium"/><meta data-rh="true" name="twitter:app:id:iphone" content="828256236"/><meta data-rh="true" property="al:ios:app_name" content="Medium"/><meta data-rh="true" property="al:ios:app_store_id" content="828256236"/><meta data-rh="true" property="al:android:package" content="com.medium.reader"/><meta data-rh="true" property="fb:app_id" content="542599432471018"/><meta data-rh="true" property="og:site_name" content="Medium"/><meta data-rh="true" property="og:type" content="article"/><meta data-rh="true" property="article:publ

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])

In [ ]:
text = [result.text for result in results]
article = ' '.join(text)

In [ ]:
article

'AIML UAE Jun 7 Tensorflow vs PyTorch: Which Will Be The Top Deep Learning Framework In 2022? **Updates** Introduction PyTorch and Tensorflow are two of the most popular deep learning libraries available today. PyTorch was developed by Facebook’s AI Research Lab to make deep neural networks easier to build and use. TensorFlow was initially authored by Google Brain Team which offers a flexible representation of data, allowing you to build custom machine learning models that range from linear regression to deep neural networks. So it’s common to hear people say. “Use [Deeplearning Library] because it’s faster or it’s more flexible.” But which one should you pick? TensorFlow was developed for internal Google use in research and production. The initial version was released in November 2015. Then Google released the updated version of TensorFlow, named TensorFlow 2.0, in September 2019 with official inbuilt Keras support. Keras is an open-source software library that provides a higher-level

## Splitting blog into smaller chunks

In [ ]:
article = article.replace('.', '.<eos>')
article = article.replace('?', '?<eos>')
article = article.replace('!', '!<eos>')

In [ ]:
sentences = article.split('<eos>')
sentences[10]

'0, in September 2019 with official inbuilt Keras support.'

In [ ]:
max_chunk = 500

In [ ]:
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
chunks[0]

'AIML UAE Jun 7 Tensorflow vs PyTorch: Which Will Be The Top Deep Learning Framework In 2022?  **Updates** Introduction PyTorch and Tensorflow are two of the most popular deep learning libraries available today.  PyTorch was developed by Facebook’s AI Research Lab to make deep neural networks easier to build and use.  TensorFlow was initially authored by Google Brain Team which offers a flexible representation of data, allowing you to build custom machine learning models that range from linear regression to deep neural networks.  So it’s common to hear people say.  “Use [Deeplearning Library] because it’s faster or it’s more flexible. ” But which one should you pick?  TensorFlow was developed for internal Google use in research and production.  The initial version was released in November 2015.  Then Google released the updated version of TensorFlow, named TensorFlow 2. 0, in September 2019 with official inbuilt Keras support.  Keras is an open-source software library that provides a h

In [ ]:
len(chunks)

4

## Summarization

In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res[0]

{'summary_text': ' PyTorch was developed by Facebook’s AI Research Lab to make deep neural networks easier to build and use . Tensorflow was developed for internal Google use in research and production . Google released the updated version of TensorFlow in September 2019 with official inbuilt Keras support .'}

In [ ]:
' '.join([summ['summary_text'] for summ in res])

' PyTorch was developed by Facebook’s AI Research Lab to make deep neural networks easier to build and use . Tensorflow was developed for internal Google use in research and production . Google released the updated version of TensorFlow in September 2019 with official inbuilt Keras support .  PyTorch is like Numpy but optimized for GPUs so it is more pythonic . TensorFlow is less Pythonic because it was a C++-based library ported to Python and it is also optimized for the GPU . In Tensorflow, you can define neural network layers using subclassing whereas in Keras you can either use the sequential API or subclassing .  PyTorch is a JavaScript library for Machine Learning that allows you to train and deploy models both in the browser and server-side with Node.js . Playground is a small but polished educational tool for understanding the basics of neural networks . TorchServe is an open-source deployment framework resulting from a collaboration between AWS and Facebook and was released in

## Output in the form of text file

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])

In [ ]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)